Word2Vec
========

Introducción
------------

Word2Vec es un popular algoritmo desarrollado por Tomáš Mikolov en varias publicaciones [Mikolov et al., 2013b,a] que utiliza el concepto de representaciones densas y distribuidas. La idea principal de Word2Vec es que el significado de una palabra puede inferirse del contexto en el que aparece. En este sentido, el texto es **simplemente una secuencia de palabras donde el significado de cada una de ellas se puede entraer de las palabras que están antes y despues de la misma**. Podríamos decir que busca codificar cada palabra utilizando el contexto en el que aparece en lugar de la palabra propiamente dicha.

    "Dime con quien andas y te diré quien eres"

Intuición
----------

Si dos palabras distintas tienden a aparecer siempre en el mismo contexto, entonces las mismas deberían de compartir un espacio similar en las representaciones. Esto quiere decir que las representaciones de todo el vocabulario generan un espacio multidimensional donde la posición relativa de cada palabra con las otras tiene significado. Más aún, habría entonces direcciones dentro del espacio con signfiicados especificos.

Definición de contexto
----------------------

La definición de contexto que utilicemos tiene un efecto importante en los resultados de los vectores que generamos para las palabras y en las propiedades que los mismos tienen. En la mayoría de los casos el contexto serán las palabras que rodean a cada palabra en cuestión - normalmente en la forma de una ventana alrededor de la misma - pero podría también ser la oración completa o el parrafo completo en donde aparece.

En algunos incluso el contexto puede ser variable, donde el mismo es inferido dependiendo de la estructura sintáctica de donde aparece la palabra.

### Ventana móvil

Quizás la estrategia más típica es la de una ventana móvil en la cual el contexto está definido por la secuencia de `2m + 1` palabras. La pabra en la posición `m + 1` es la palabra *foco*, y las restantes palabras son las palabras *contexto*.

El tamaño de la ventana tiene un efecto importante. Ventanas muy grandes tienden a generar directamente *tópicos* similares mientras que ventanas más pequeñas tenderán a generar vectores que son más funcionales o sintácticas.

### Ventana móvil posicional

El concepto de contexto utilizando una ventana movil hace que las palabras pierdan la noción del orden. Ni siquiera tenemos información de las palabras que aparecen antes o despues de la palabra foco. Esto se puede resolver utilizando contextos posicionales donde en lugar de codificar la palabra "perro" por ejemplo, codificamos "perro:+2" haciendo referencia a que la misma está 2 posiciones luego de la palabra foco.


¿Cómo aprender estas representaciones?
--------------------------------------

La forma en la que Word2Vec genera estas representaciones es a través del modelado de una tarea de lenguaje especifica que es aprender una distribución probabilistica de palabras dado un contexto. Es decir que contruiremos un modelo que nos pueda decir, por cada palabra, la probabilidad de que una palabra dada aparezca en su contexto. Por ejemplo, dada la palabra "sobiética" la probabilidad de que la palabra "unión" o "Rusia" esté en el contexto es mucho más alta que la palabra "perro".

Esto lo podemos lograr de dos formas:

### Skip-grams (SG)

En esta modalidad, la tarea a resolver consiste en dada una palabra y su contexto en el que aparece, por cada palabra en el contexto predecir cada una de las restantes palabras que la rodean.

![Skip-grams](https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/word2vec_sg.png?raw=1)

Esta variante desacopla las dependencias entre el contexto y cada palabra al asumir que cada elemento del contexto es independiente de cada uno. Esto quiere decir, que nuestro set de datos para resolver la tarea luciría algo como lo siguiente:

![Skip-grams: conjunto de entrenamiento](https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/word2vec_sg_train.png?raw=1)

La técnica de skip-gram fué la popularizada por Mikolov et al. [2013a] y es una de las técnicas más eficientes y róbustas para aprender vectores.

### Continous-bag-of-words (CBOW)

En esta modalidad, la tarea a resolver consiste en dada un contexto, predecir cada una de las palabras que están dentro del contexto utilizando las palabras restantes.

![Continous-bag-of-words](https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/word2vec_cbow.png?raw=1)

En esta variante el contexto se define como la suma de todas los vectores palabra, aunque perdiendo la información del orden. Esto quiere decir, que nuestro set de datos para resolver la tarea luciría algo como lo siguiente:

![Continous-bag-of-words: conjunto de entrenamiento](https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/word2vec_cbow_train.png?raw=1)

Entrenamiento
-------------

En cualquiera de las anteriores modalidades, nuestro modelo será una red neuronal que tendrá una sola capa interna (hidden layer) donde la cantidad de unidades de la dicha capa está dada por la dimensionalidad de los vectores que queremos aprender. En el caso de Word2Vec, esta dimensionalidad es 300. Estos vectores los debemos generar para todas las palabras del vocabulario, por lo tanto nuestra capa interna está representada por una matriz de dimensiones *|V| x 300* (una columna por cada unidad neuronal).

Al final del proceso, nuestro objetivo entonces es aprender esta matriz de pesos en la red neuronal, con la esperanza de que los vectores que se aprendieron hayan captura de alguna forma la distribución probabilistica de cada una de las palabras. Si nos ponemos a revisar esta configuración, la matriz resultante no es más que una tabla de búsqueda de vectores, que dada una palabra de nuestro vocabulario, nos devuelve el vector resultante.

<img src="https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/word2vec_lookup.png?raw=1" width="500" style="margin: 0 auto 0" />

Modelo pre-entrenado en Español
-------------------------------

Utilizaremos un modelo pre-entrenado de Word2Vec que fué entrenado con un corpus en español (Spanish CoNLL17 corpus). Este modelo se entrenó trabajando con una ventana de 10 palabras, con el algoritmo Continuous bag of words y un negative-sampling de 5 palabras. Este modelo está almacenado en formato binario, por lo que lo cargamos utilizando el objeto `KeyedVectors` y el metodo `load_word2vec_format`:

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/vectorization/Word2Vec.txt \
    --quiet --no-clobber
!pip install -r Word2Vec.txt --quiet

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Descargar el modelo

El siguiente fragmento de codigo descarga el modelo preentrenado y lo almacena en el directorio `Models/Word2Vec`

In [3]:
!mkdir -p ./Models/Word2Vec
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model-es.bin \
    --quiet --no-clobber

Para comenzar a utilizar este modelo, lo cargaremos desde el directorio anterior:

In [4]:
import sys, os, io, pathlib
import numpy as np

from gensim.models import KeyedVectors
from gensim.test.utils import datapath

path = os.path.abspath('/content/model-es.bin')
embeddings = KeyedVectors.load_word2vec_format(datapath(path), binary=True)

Si exploramos algunas de las propiedades del modelo veremos el tamaño del vocabulario y las dimensiones de nuestros vectores resultantes:

In [5]:
print("El tamaño del vocabulario es:", len(embeddings.key_to_index))
print("El tamaño de los vectores es:", embeddings.vector_size)

El tamaño del vocabulario es: 2656057
El tamaño de los vectores es: 100


### Explorando las representaciones

Podemos comenzar a indagar como lucen las representaciones vectoriales que se aprendieron utilizando el método most_similar. Este método nos devuelve las 10 primeras palabras más similares a la palabra que indicamos. La similaridad se computa utilizando cosine-similarity de las representaciones vectoriales de cada una de las palabras.

In [6]:
embeddings.most_similar("reina")

[('emperatriz', 0.7729766964912415),
 ('princesa', 0.7713199853897095),
 ('sofía', 0.7699437141418457),
 ('coronaciónla', 0.7649693489074707),
 ('sofíala', 0.7604084610939026),
 ('virreina', 0.7558673024177551),
 ('categoría:talavera', 0.7512168288230896),
 ('isabel', 0.7497586607933044),
 ('sofíael', 0.7353644967079163),
 ('emperatríz', 0.7345834374427795)]

También podemos consultar cuales son las palabras más disimilares a una determinada palabra. Esto lo hacemos especificando el parametro negative dentro del método `most_similar`

In [7]:
embeddings.most_similar(negative="reina")

[('categoría:triplemanía', 0.31081730127334595),
 ('smr10', 0.30711960792541504),
 ('ausespecial=sub20', 0.3018966019153595),
 ('categoría:adiscus', 0.3002437353134155),
 ('aus17', 0.2890426814556122),
 ('categoría:agavoideae', 0.2831123173236847),
 ('€463.520', 0.280171275138855),
 ('categoría:maturinenses', 0.2768503427505493),
 ('categoría:errázuriz', 0.2763756513595581),
 ('categoría:odesitas', 0.2753686308860779)]

### Analogias

El método anterior no es muy util en general, sin embargo nos permite introducir el concepto de aritmética sobre estos mismos vectores. Por ejemplo: Pordiamos consultar cual es la analogia de dos palabras. El siguiente ejemplo se leería así: A lo que Paris es a Francia, ¿cúal es la analogia de Madrid?. Esto se resolveria tomando "Francia", quitandole "Paris" y agregandole "Madrid"

<img src="https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/word2vec_vector_space.png?raw=1" width="500" />

In [ ]:
embeddings.most_similar(positive=["francia", "madrid"], negative=["paris"])[0][0]

'españa'

In [ ]:
embeddings.most_similar(positive=["paris", "españa"], negative=["francia"])[0][0]

'barcelona'

In [9]:
embeddings.most_similar(positive=["paris", "españa"], negative=["francia"])[0:2]

[('barcelona', 0.7425479888916016), ('madrid', 0.7323868274688721)]

Estos nos dice que quizás este espacio vectorial no es solamente un espacio donde vectores que están cerca los unos de los otros en el espacio tienen significados similares, sino que en realidad capturan el significado en una forma más produnda. Concretamente, que hay "direcciones de significados" en el espacio donde uno de puede mover. Veamos algunos ejemplos:

En la dirección de "hacedor"
 - Conducir es a conductor lo que limpiar es a?

In [ ]:
embeddings.most_similar(positive=["conductor", "limpiar"], negative=["conducir"])[0][0]

'trapeador'

En la dirección "bebidas"
 - El vino es a Francia lo que el whisky es a?

In [ ]:
embeddings.most_similar(positive=["francia", "whisky"], negative=["vino"])[0][0]

'bélgica'

En la dirección "extremar"
 - Bueno es a genial lo que malo es a?

In [ ]:
embeddings.most_similar(positive=["buenisimo", "malo"], negative=["bueno"])[0][0]

'malisimo'

Visualización del espacio contino de word2vec
---------------------------------------------

El siguiente código utiliza un plugin de Tensorflow llamado Tensorboard que permite hacer una proyección en un espacio 3D (para luego hacer una proyección en un espacio 2D - el de la pantalla) de las representaciones de cada una de las palabras que están en el vocabulario. Esta visualización requiere tener instalado TensorFlow 2.0

In [ ]:
!mkdir -p '/tmp/logdir'

In [ ]:
import tensorflow as tf
from tqdm import tqdm
from tensorboard.plugins import projector

logdir = '/tmp/logdir'

In [ ]:
with tf.device('/CPU:0'):
    weights = tf.Variable(embeddings.vectors, )
    checkpoint = tf.train.Checkpoint(embedding=weights)
    checkpoint.save(os.path.join(logdir, 'embedding.ckpt'))

2021-09-15 19:11:52.646960: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-15 19:11:52.652627: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-15 19:11:52.655446: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


El siguiente codigo genera los "labels" asociados al espacio vectorial que cargamos anteriormente. Pueden descargar estos labels ya preprocesados desde:

In [ ]:
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/meta.tsv \
    --quiet --no-clobber --directory-prefix '/tmp/logdir'

In [ ]:
os.path.exists(logdir + '/vecs.tsv')

True

In [ ]:
if os.path.exists(logdir + '/vecs.tsv') == False:
    with io.open(logdir + '/meta.tsv', 'w', encoding='utf-8') as metadata_file:
        with io.open(logdir + '/vecs.tsv', 'w', encoding='utf-8') as vectors_file:
            for index in tqdm(range(len(embeddings.index_to_key))):
                word = embeddings.index_to_key[index]
                vec = embeddings.vectors[index]
                metadata_file.write(word + "\n")
                vectors_file.write('\t'.join([str(x) for x in vec]) + "\n")
else:
    print('File vecs.tsv is already generated')

File vecs.tsv is already generated


In [ ]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'meta.tsv'
projector.visualize_embeddings(logdir, config)

Ejecute en una consola:

```
tensorboard --logdir /tmp/logdir
```

Debería poder visualizar en el navegador el espacio vectorial de las diferentes palabras capturadas por el modelo de Word2Vec. Pruebe buscando una palabra en la barra de navegación lateral, como por ejemplo "avenida" para ver cuales son las palabras cercanas dentro del espacio vectorial. Utilice el selector `neighbors` para limitar la cantidad de vecinos más próximos que se muestran en la pantalla.

![Espacio vectorial para la palabra avenida](https://github.com/santiagxf/M72109/blob/master/docs/nlp/_images/word2vec_tensorboard.png?raw=1)